In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd drive/My\ Drive/Colab/YOLO-V3-Keras-from-scratch/

/content/drive/My Drive/Colab/YOLO-V3-Keras-from-scratch


In [3]:
import os
import cv2
import json
import random
import colorsys

import numpy as np
import tensorflow as tf

from   keras.models               import Sequential, Model ,model_from_yaml ,load_model
from   keras.activations          import linear
from   keras.layers.normalization import BatchNormalization
from   keras.optimizers           import Adagrad
from   keras.utils                import plot_model
from   keras.engine.topology      import Layer
from   yolo3.model                import yolo_eval
from   keras                      import backend as K

Using TensorFlow backend.


In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX
weights_path   = '../../yolov3_weights.h5'
anchors_path = 'model_data/yolo_anchors.txt'
classes_path = 'model_data/coco_classes.txt'
score = 0.3
iou = 0.5

In [ ]:
def _get_class():
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

In [ ]:
def _get_anchors():
    with open(anchors_path) as f:
        anchors = f.readline()
        anchors = [float(x) for x in anchors.split(',')]
        anchors = np.array(anchors).reshape(-1, 2)
    return anchors

In [ ]:
def generate():
    assert weights_path.endswith('.h5'), 'Keras model must be a .h5 file.'
    print('{} model, anchors, and classes loaded.'.format(weights_path))
    yaml_file = open('infer_model.yaml', 'r')
    loaded_model_yaml = yaml_file.read()
    yaml_file.close()
    Infer_model = model_from_yaml(loaded_model_yaml)
    Infer_model.load_weights(weights_path)
    # Generate output tensor targets for filtered bounding boxes.
    input_image_shape = K.placeholder(shape=(2, ))
    boxes, scores, classes = yolo_eval(Infer_model.output, anchors,
                                       len(class_names), input_image_shape,
                                       score_threshold=score, iou_threshold=iou)
    return Infer_model,boxes, scores, classes,input_image_shape

In [ ]:
def close_session(self):
    sess.close()

In [ ]:
def detect_image(image,Infer_model,input_image_shape, database=None):
    boxed_image = cv2.resize(image,model_image_size)
    image_data  = np.array(boxed_image, dtype='float32')
    image_data /= 255.
    image_data = np.expand_dims(image_data, 0)  # Add batch dimension.
    out_boxes, out_scores, out_classes = sess.run(
        [boxes, scores, classes],
        feed_dict={
            Infer_model.input: image_data,
            input_image_shape: [boxed_image.shape[0], boxed_image.shape[1]],
            K.learning_phase(): 0
        })
    return out_boxes, out_scores, out_classes

In [10]:
class_names = _get_class()
anchors = _get_anchors()
sess = K.get_session()
hsv_tuples = [(x / len(class_names), 1., 1.)
              for x in range(len(class_names))]
colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
colors = list(map(lambda x: (int(x[0] * 255), int(x[1] * 255),
                             int(x[2] * 255)),colors))
random.seed(10101)  # Fixed seed for consistent colors across runs.
# Shuffle colors to decorrelate adjacent classes.
random.shuffle(colors)
random.seed(None)  # Reset seed to default.
model_image_size = (416, 416)  # fixed size or (None, None)
Infer_model,boxes, scores, classes,input_image_shape = generate()

../../yolov3_weights.h5 model, anchors, and classes loaded.


In [ ]:
cap    = cv2.VideoCapture('tt2.avi')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
width  = cap.get(cv2.CAP_PROP_FRAME_WIDTH)   # float
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT) # float
out    = cv2.VideoWriter('tt2.mp4',fourcc, 30.0, (int(width),int(height)))
label_size =int((width + height) // 300)
he=height/416
wi=width /416
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        out_boxes, out_scores, out_classes=detect_image(frame,Infer_model,input_image_shape)
        for i, c in reversed(list(enumerate(out_classes))):
            predicted_class = class_names[c]
            box = out_boxes[i]
            score = round(float(out_scores[i]),2)
            label = '{} {:.2f}'.format(predicted_class, score)
            top, left, bottom, right = box
            top    = int(max(0, np.floor(he*top + 0.5).astype('int32')))
            left   = int(max(0, np.floor(wi*left + 0.5).astype('int32')))
            bottom = int(min(height, np.floor(he*bottom + 0.5).astype('int32')))
            right  = int(min(width, np.floor(wi*right + 0.5).astype('int32')))
            if top - label_size >= 0:
                text_origin = np.array([left, top - label_size])
            else:
                text_origin = np.array([left, top + 1])
            cv2.rectangle(frame,(left,top),(right,bottom),colors[c],3)    
            cv2.putText(frame,label,tuple(text_origin),font,0.5,colors[c])
        out.write(frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
cap.release()
out.release()
cv2.destroyAllWindows()